In [ ]:
# import required modules
import os
import arcpy
arcpy.env.overwriteOutput = True

dir_root = r'E:\\ChlorA\\PML\\monthly\\chlor_a\\'
years = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]

# [print(x) for x in years]
for (x) in years:
    print(x)
    directory = str(dir_root) + str((x)) + str('\\')
    print(directory)
## iterate over files in that directory
    for nc_file in os.scandir(directory):
     if nc_file.is_file():
         print(nc_file.path)
         nc_filename = (os.path.basename(nc_file).split('\\')[-1])
         print(nc_filename)
         tif_filename = r'E:\\ChlorA\\PML\\6.0\\' + nc_filename.split('.',1)[0] + ".0.tif"
         print(tif_filename)
         ras = "in_memory/ras"
         print("Processing: " + nc_filename)
         arcpy.md.MakeNetCDFRasterLayer(nc_file.path, "chlor_a", "lon", "lat", ras, '', '', "BY_VALUE", "CENTER")
         arcpy.Raster(ras).save(tif_filename)

In [ ]:
sub2_dir_root = r'E:\\ChlorA\\NESDIS\\coastwatch\\viirs\\science\\L3\\global\\chlora\\anom\\WW00\\'
sub2_years = [2018,2019,2020,2021,2022,2023,2024]

for (x) in sub2_years:
    print(x)
    directory = str(sub2_dir_root) + str((x)) + str('\\')
    print(directory)
    for sub2_nc_file in os.scandir(directory):
     if sub2_nc_file.is_file() and sub2_nc_file.name.endswith('pdif.nc'):
         print(sub2_nc_file.path)
         sub2_nc_filename = (os.path.basename(sub2_nc_file).split('\\')[-1])
         print(sub2_nc_filename)
         tif_filename = r'E:\\ChlorA\\NESDIS\\2024_Reporting\\' + sub2_nc_filename.split('.',1)[0] + ".tif"
         print(tif_filename)
         ras = "in_memory/ras"
         arcpy.md.MakeNetCDFRasterLayer(sub2_nc_file.path, "chlor_a_pdif", "lon", "lat", ras, '', None, "BY_VALUE", "CENTER")
         arcpy.Raster(ras).save(tif_filename)
         arcpy.management.BuildPyramids(tif_filename,-1, "NONE", "NEAREST", "DEFAULT", 75, "OVERWRITE")
         con_raster = arcpy.sa.Con(tif_filename, tif_filename, None, "VALUE >= -20 And VALUE <= 20")
         con_raster.save(tif_filename.split(".")[0] + "_con.tif")
         arcpy.management.Delete(tif_filename)

In [23]:
## add PML rasters to a new geodatabase and mosaic
processing_dir = r"E:\ChlorA\Processing\Reporting2024"
processing_gdbName = "Processing2024"
processing_gdb = processing_dir + '\\' + processing_gdbName + '.gdb'

arcpy.management.CreateFileGDB(
    out_folder_path=processing_dir,
    out_name=processing_gdbName,
    out_version="CURRENT"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb'>

In [24]:
mosaic_name = "sub1_chlor_a_monthly_1997to2023_PMLv6_0"
mosaic_path = processing_gdb + '\\' + mosaic_name

arcpy.management.CreateMosaicDataset(
    in_workspace=processing_gdb,
    in_mosaicdataset_name=mosaic_name,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub1_chlor_a_monthly_1997to2023_PMLv6_0'>

In [25]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=mosaic_path,
    raster_type="Raster Dataset",
    input_path=r"E:\ChlorA\Processing\Reporting2024\PML",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="OVERWRITE_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\Administrator\AppData\Local\ESRI\rasterproxies\sub1_chlor_a_monthly_1997to2023_PMLv6_0"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub1_chlor_a_monthly_1997to2023_PMLv6_0'>

In [27]:
mosaic_name = "sub2_chlor_a_daily_pdif_2018to2023_VIIRS"
VIIRSmosaic_path = processing_gdb + '\\' + mosaic_name

arcpy.management.CreateMosaicDataset(
    in_workspace=processing_gdb,
    in_mosaicdataset_name=mosaic_name,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [29]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=VIIRSmosaic_path,
    raster_type="Raster Dataset",
    input_path=r"E:\ChlorA\NESDIS\2024_Reporting",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="OVERWRITE_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\Administrator\AppData\Local\ESRI\rasterproxies\sub2_chlor_a_daily_pdif_2018to2023_VIIRS")

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [64]:
from datetime import datetime

name = 'V2018001_D1_WW00_chlorapdif_con'
day_num = (int(name[5:8]))
year_num = (int(name[1:5]))
res = datetime.strptime(str(year_num) + "-" + str(day_num), "%Y-%j").strftime("%m-%d-%Y")
print(res)

2018
001
01-01-2018


In [67]:
arcpy.management.CalculateField(
    in_table=VIIRSmosaic_path,
    field="acquisition_date",
    expression="calc_date(!Name!)",
    expression_type="PYTHON3",
    code_block="""def calc_date(Name):
    day_num = (int(Name[5:8]))
    year_num = (int(Name[1:5]))
    esri_date = datetime.strptime(str(year_num) + "-" + str(day_num), "%Y-%j").strftime("%m/%d/%Y")
    return(esri_date)
    """,
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [68]:
arcpy.management.CalculateField(
    in_table=VIIRSmosaic_path,
    field="variable",
    expression='"chlor-a pdif"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [53]:
arcpy.management.CalculateField(
    in_table=mosaic_path,
    field="variable",
    expression='"chlorophyll-a"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub1_chlor_a_monthly_1997to2023_PMLv6_0'>

In [55]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset=mosaic_path,
    variable_field="variable",
    dimension_fields="acquisition_date acquisition_month Months",
    variable_desc_units="chlorophyll-a 'Chlorophyll-a concentration in seawater' 'milligram m-3'",
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub1_chlor_a_monthly_1997to2023_PMLv6_0'>

In [69]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset=VIIRSmosaic_path,
    variable_field="variable",
    dimension_fields="acquisition_date acquisition_day Days",
    variable_desc_units="chlor-a pdif 'Ratio of chlorophyll-a anomaly over 61 day composite' 'ratio'",
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [ ]:
arcpy.management.AddIndex(
    in_table=mosaic_path,
    fields="StdTime",
    index_name="Time_Index",
    unique="NON_UNIQUE",
    ascending="NON_ASCENDING"
)

In [70]:
arcpy.management.AddIndex(
    in_table=VIIRSmosaic_path,
    fields="StdTime",
    index_name="Time_Index",
    unique="NON_UNIQUE",
    ascending="NON_ASCENDING"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\Processing2024.gdb\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS'>

In [71]:
print(processing_dir + '\\CRF\\' + mosaic_name + ".crf")

E:\ChlorA\Processing\Reporting2024\CRF\sub2_chlor_a_daily_pdif_2018to2023_VIIRS.crf


In [57]:
PMLcrf_name = str(processing_dir + '\\CRF\\' + mosaic_name + ".crf")
arcpy.management.CopyRaster(
    in_raster=mosaic_path,
    out_rasterdataset=PMLcrf_name,
    config_keyword="",
    background_value=None,
    nodata_value="",
    onebit_to_eightbit="NONE",
    colormap_to_RGB="NONE",
    pixel_type="32_BIT_FLOAT",
    scale_pixel_value="NONE",
    RGB_to_Colormap="NONE",
    format="CRF",
    transform="NONE",
    process_as_multidimensional="ALL_SLICES",
    build_multidimensional_transpose="TRANSPOSE"
)

<Result 'E:\\ChlorA\\Processing\\Reporting2024\\CRF\\sub2_chlor_a_daily_pdif_2018to2023_VIIRS.crf'>

In [72]:
PMLcrf_name = str(processing_dir + '\\CRF\\' + mosaic_name + ".crf")
arcpy.management.CopyRaster(
    in_raster=VIIRSmosaic_path,
    out_rasterdataset=PMLcrf_name,
    config_keyword="",
    background_value=None,
    nodata_value="",
    onebit_to_eightbit="NONE",
    colormap_to_RGB="NONE",
    pixel_type="32_BIT_FLOAT",
    scale_pixel_value="NONE",
    RGB_to_Colormap="NONE",
    format="CRF",
    transform="NONE",
    process_as_multidimensional="ALL_SLICES",
    build_multidimensional_transpose="TRANSPOSE"
)